In [ ]:
from grid.eval import *

In [ ]:
boardsize = 3
n_workers = 8

snaps = snapshots(boardsize)
snaps = pd.concat([snaps, parameters(snaps)], 1)
snaps['nickname'] = snaps.run.str.extract('.* (.*)', expand=False) + '.' + snaps.idx.astype(str)
snaps['params'] = params(snaps)
snaps = snaps.set_index('nickname')

n_agents = len(snaps)
wins  = pd.DataFrame(np.zeros((n_agents, n_agents)), snaps.index, snaps.index)
games = pd.DataFrame(np.zeros((n_agents, n_agents)), snaps.index, snaps.index)

soln = None

In [ ]:
futures = {}
with ProcessPoolExecutor(n_workers) as pool:
    while True:
        if len(futures) < n_workers:
            try:
                soln = activelo.solve(games, wins, soln=soln)
                sugg = suggest(soln)
            except InManifoldError:
                soln = None
                sugg = tuple(np.random.choice(snaps.index, (2,)))
                log.warning('Got a manifold error; making a random suggestion')
            futures[sugg] = pool.submit(evaluate, *sugg)
        for key, future in list(futures.items()):
            if future.done():
                results = future.result()
                games, wins = update(games, wins, results)
                del futures[key]
    
        if soln is not None:
            report(soln)
            _, σ = arena.analysis.difference(soln, soln.μ.idxmin())
            if σ.pow(2).mean()**.5 < .1:
                break
        
        time.sleep(1)

snaps['μ'], snaps['σ'] = arena.analysis.difference(soln, soln.μ.idxmin())

In [ ]:
n_agents = len(agents)
wins  = pd.DataFrame(np.zeros((n_agents, n_agents)), list(agents), list(agents))
games = pd.DataFrame(np.zeros((n_agents, n_agents)), list(agents), list(agents))
soln = None